In [5]:
! pip -q install torchtext==0.6.0
! pip -q install pyvi 

import nltk
nltk.download('wordnet')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.1 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is inc

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import os
import math

# Kiến trúc mô hình

In [7]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        
        self.embed = nn.Embedding(vocab_size, d_model)
        
    def forward(self, x):
        return self.embed(x)
        

In [8]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_length=200, dropout=0.1):
        super().__init__()
        
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(max_seq_length, d_model)
        
        # Bảng pe mình vẽ ở trên 
        for pos in range(max_seq_length):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos/(10000**(2*i/d_model)))
                pe[pos, i+1] = math.cos(pos/(10000**((2*i+1)/d_model)))
        pe = pe.unsqueeze(0)        
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        
        x = x*math.sqrt(self.d_model)
        seq_length = x.size(1)
        
        pe = Variable(self.pe[:, :seq_length], requires_grad=False)
        
        if x.is_cuda:
            pe.cuda()
        # cộng embedding vector với pe 
        x = x + pe
        x = self.dropout(x)
        
        return x
    

In [9]:
def attention(q, k, v, mask=None, dropout=None):
    """
    q: batch_size x head x seq_length x d_model
    k: batch_size x head x seq_length x d_model
    v: batch_size x head x seq_length x d_model
    mask: batch_size x 1 x 1 x seq_length
    output: batch_size x head x seq_length x d_model
    """

    # attention score được tính bằng cách nhân q với k
    d_k = q.size(-1)
    scores = torch.matmul(q, k.transpose(-2, -1))/math.sqrt(d_k)
    
    if mask is not None:
        mask = mask.unsqueeze(1)
        scores = scores.masked_fill(mask==0, -1e9)
    # xong rồi thì chuẩn hóa bằng softmax
    scores = F.softmax(scores, dim=-1)
    
    if dropout is not None:
        scores = dropout(scores)
    
    output = torch.matmul(scores, v)
    return output, scores


In [10]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout=0.1):
        super().__init__()
        assert d_model % heads == 0
        
        self.d_model = d_model
        self.d_k = d_model//heads
        self.h = heads
        self.attn = None

        # tạo ra 3 ma trận trọng số là q_linear, k_linear, v_linear như hình trên
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        """
        q: batch_size x seq_length x d_model
        k: batch_size x seq_length x d_model
        v: batch_size x seq_length x d_model
        mask: batch_size x 1 x seq_length
        output: batch_size x seq_length x d_model
        """
        bs = q.size(0)
        # nhân ma trận trọng số q_linear, k_linear, v_linear với dữ liệu đầu vào q, k, v 
        # ở bước encode các bạn lưu ý rằng q, k, v chỉ là một (xem hình trên)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)
        
        # tính attention score
        scores, self.attn = attention(q, k, v, mask, self.dropout)
        
        concat = scores.transpose(1, 2).contiguous().view(bs, -1, self.d_model)
        
        output = self.out(concat)
        return output


In [11]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model
        
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        
        self.eps = eps
    
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

In [12]:
class FeedForward(nn.Module):
    """ Trong kiến trúc của chúng ta có tầng linear 
    """
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
    
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

In [13]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model, dropout=dropout)
        self.ff = FeedForward(d_model, dropout=dropout)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        """
        x: batch_size x seq_length x d_model
        mask: batch_size x 1 x seq_length
        output: batch_size x seq_length x d_model
        """
        
        
        x2 = self.norm_1(x)
        # tính attention value, các bạn để ý q, k, v là giống nhau        
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x


In [14]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(heads, d_model, dropout=dropout)
        self.attn_2 = MultiHeadAttention(heads, d_model, dropout=dropout)
        self.ff = FeedForward(d_model, dropout=dropout)

    def forward(self, x, e_outputs, src_mask, trg_mask):
        """
        x: batch_size x seq_length x d_model
        e_outputs: batch_size x seq_length x d_model
        src_mask: batch_size x 1 x seq_length
        trg_mask: batch_size x 1 x seq_length
        """
        # Các bạn xem hình trên, kiến trúc mình vẽ với code ở chỗ này tương đương nhau.
        x2 = self.norm_1(x)
        # multihead attention thứ nhất, chú ý các từ ở target 
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        # masked mulithead attention thứ 2. k, v là giá trị output của mô hình encoder
        x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs, src_mask))
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x


Cài đặt Encoder
bao gồm N encoder layer

In [15]:
import copy

def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class Encoder(nn.Module):
    """Một encoder có nhiều encoder layer nhé !!!
    """
    def __init__(self, vocab_size, d_model, N, heads, dropout):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model, dropout=dropout)
        self.layers = get_clones(EncoderLayer(d_model, heads, dropout), N)
        self.norm = Norm(d_model)
        
    def forward(self, src, mask):
        """
        src: batch_size x seq_length
        mask: batch_size x 1 x seq_length
        output: batch_size x seq_length x d_model
        """
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, mask)
        return self.norm(x)


Cài đặt Decoder
bao gồm N decoder layers

In [16]:
class Decoder(nn.Module):
    """Một decoder có nhiều decoder layer nhé !!!
    """
    def __init__(self, vocab_size, d_model, N, heads, dropout):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model, dropout=dropout)
        self.layers = get_clones(DecoderLayer(d_model, heads, dropout), N)
        self.norm = Norm(d_model)
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        """
        trg: batch_size x seq_length
        e_outputs: batch_size x seq_length x d_model
        src_mask: batch_size x 1 x seq_length
        trg_mask: batch_size x 1 x seq_length
        output: batch_size x seq_length x d_model
        """
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)
    

Cài đặt Transformer 
bao gồm encoder và decoder

In [17]:
class Transformer(nn.Module):
    """ Cuối cùng ghép chúng lại với nhau để được mô hình transformer hoàn chỉnh
    """
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads, dropout):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads, dropout)
        self.decoder = Decoder(trg_vocab, d_model, N, heads, dropout)
        self.out = nn.Linear(d_model, trg_vocab)
    def forward(self, src, trg, src_mask, trg_mask):
        """
        src: batch_size x seq_length
        trg: batch_size x seq_length
        src_mask: batch_size x 1 x seq_length
        trg_mask batch_size x 1 x seq_length
        output: batch_size x seq_length x vocab_size
        """
        e_outputs = self.encoder(src, src_mask)
        
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output
        

Load dữ liệu


Chúng ta sử dụng torchtext để load dữ liệu, giúp giảm thời gian và hiệu quả 

In [18]:
from torchtext import data

class MyIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

global max_src_in_batch, max_tgt_in_batch

def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)
    

In [19]:
def nopeak_mask(size, device):
    """Tạo mask được sử dụng trong decoder để lúc dự đoán trong quá trình huấn luyện
     mô hình không nhìn thấy được các từ ở tương lai
    """
    np_mask = np.triu(np.ones((1, size, size)),
    k=1).astype('uint8')
    np_mask =  Variable(torch.from_numpy(np_mask) == 0)
    np_mask = np_mask.to(device)
    
    return np_mask

def create_masks(src, trg, src_pad, trg_pad, device):
    """ Tạo mask cho encoder, 
    để mô hình không bỏ qua thông tin của các kí tự PAD do chúng ta thêm vào 
    """
    src_mask = (src != src_pad).unsqueeze(-2)

    if trg is not None:
        trg_mask = (trg != trg_pad).unsqueeze(-2)
        size = trg.size(1) # get seq_len for matrix
        np_mask = nopeak_mask(size, device)
        if trg.is_cuda:
            np_mask.cuda()
        trg_mask = trg_mask & np_mask
        
    else:
        trg_mask = None
    return src_mask, trg_mask
    

In [20]:
import nltk
import re
from nltk.tokenize import word_tokenize

class tokenize(object):
    
    def __init__(self, lang=None):
        pass  
            
    def tokenizer(self, sentence):
        sentence = re.sub(
            r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(sentence))
        sentence = re.sub(r"[ ]+", " ", sentence)
        sentence = re.sub(r"\!+", "!", sentence)
        sentence = re.sub(r"\,+", ",", sentence)
        sentence = re.sub(r"\?+", "?", sentence)
        sentence = sentence.lower()
        return word_tokenize(sentence)


Data loader
Sử dụng torchtext để load dữ liệu nhanh chóng

In [21]:
import os
import dill as pickle
import pandas as pd

def read_data(src_file, trg_file):
    src_data = open(src_file).read().strip().split('\n')

    trg_data = open(trg_file).read().strip().split('\n')
 
    return src_data, trg_data

def create_fields(src_lang, trg_lang):
    t_src = tokenize(src_lang)
    t_trg = tokenize(trg_lang)

    TRG = data.Field(lower=True, tokenize=t_trg.tokenizer, init_token='<sos>', eos_token='<eos>')
    SRC = data.Field(lower=True, tokenize=t_src.tokenizer)
        
    return SRC, TRG

def create_dataset(src_data, trg_data, max_strlen, batchsize, device, SRC, TRG, istrain=True):

    print("Creating dataset and Iterator... ")

    raw_data = {'src' : [line for line in src_data], 'trg': [line for line in trg_data]}
    df = pd.DataFrame(raw_data, columns=["src", "trg"])
    
    mask = (df['src'].str.count(' ') < max_strlen) & (df['trg'].str.count(' ') < max_strlen)
    df = df.loc[mask]

    df.to_csv("translate_transformer_temp.csv", index=False)
    
    data_fields = [('src', SRC), ('trg', TRG)]
    train = data.TabularDataset('./translate_transformer_temp.csv', format='csv', fields=data_fields)

    train_iter = MyIterator(train, batch_size=batchsize, device=device,
                        repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                        batch_size_fn=batch_size_fn, train=istrain, shuffle=True)
    
    os.remove('translate_transformer_temp.csv')
    
    if istrain:
        SRC.build_vocab(train)
        TRG.build_vocab(train)

    return train_iter


In [22]:
def step(model, optimizer, batch, criterion):
    """
    Một lần cập nhật mô hình
    """
    model.train()

    # Chuyển dữ liệu vào device (CUDA)
    src = batch.src.transpose(0, 1).cuda()
    trg = batch.trg.transpose(0, 1).cuda()
    trg_input = trg[:, :-1]
    
    # Tạo mask cho source và target
    src_mask, trg_mask = create_masks(src, trg_input, src_pad, trg_pad, opt['device'])
    
    # Dự đoán từ mô hình
    preds = model(src, trg_input, src_mask, trg_mask)
    
    # Định dạng lại target
    ys = trg[:, 1:].contiguous().view(-1)

    # Zero gradients từ optimizer
    optimizer.zero_grad()
    
    # Tính toán loss
    loss = criterion(preds.view(-1, preds.size(-1)), ys)
    
    # Backpropagate lỗi
    loss.backward()
    
    # Cập nhật các tham số của mô hình
    optimizer.step()


    loss = loss.item()
    
    return loss


In [23]:
def validiate(model, valid_iter, criterion):
    """ Tính loss trên tập validation
    """
    model.eval()
    
    with torch.no_grad():
        total_loss = []
        for batch in valid_iter:
            src = batch.src.transpose(0,1).cuda()
            trg = batch.trg.transpose(0,1).cuda()
            trg_input = trg[:, :-1]
            src_mask, trg_mask = create_masks(src, trg_input, src_pad, trg_pad, opt['device'])
            preds = model(src, trg_input, src_mask, trg_mask)

            ys = trg[:, 1:].contiguous().view(-1)
            
            loss = criterion(preds.view(-1, preds.size(-1)), ys)
            
            loss = loss.item()
            
            total_loss.append(loss)
        
    avg_loss = np.mean(total_loss)
    
    return avg_loss
    

# Tạo các file data

In [24]:
%ls /kaggle/input/

mt-dataset/  weight-mt-scatch-trans/


In [26]:
import os

# Đường dẫn input
en_path = '/kaggle/input/mt-dataset/en_sents_dataset.txt'
vi_path = '/kaggle/input/mt-dataset/vi_sents_dataset.txt'

# Tạo thư mục output
os.makedirs('./data', exist_ok=True)

# Số câu cho tập valid
valid_size = 500

# Đọc dữ liệu
with open(en_path, 'r', encoding='utf-8') as f:
    en_lines = f.readlines()
with open(vi_path, 'r', encoding='utf-8') as f:
    vi_lines = f.readlines()

# Kiểm tra số lượng câu khớp nhau
assert len(en_lines) == len(vi_lines), "Hai file phải có cùng số lượng câu!"

# Ghi file train
with open('./data/train.en', 'w', encoding='utf-8') as f:
    f.writelines(en_lines[valid_size:])
with open('./data/train.vi', 'w', encoding='utf-8') as f:
    f.writelines(vi_lines[valid_size:])

# Ghi file valid
with open('./data/valid.en', 'w', encoding='utf-8') as f:
    f.writelines(en_lines[:valid_size])
with open('./data/valid.vi', 'w', encoding='utf-8') as f:
    f.writelines(vi_lines[:valid_size])

print(f"""
Đã tạo thành công:
- ./data/train.en ({len(en_lines)-valid_size} câu)
- ./data/train.vi ({len(vi_lines)-valid_size} câu)
- ./data/valid.en ({valid_size} câu)
- ./data/valid.vi ({valid_size} câu)
""")


Đã tạo thành công:
- ./data/train.en (745890 câu)
- ./data/train.vi (745890 câu)
- ./data/valid.en (500 câu)
- ./data/valid.vi (500 câu)



In [27]:
opt = {
    'train_src_data':'./data/train.en',
    'train_trg_data':'./data/train.vi',
    'valid_src_data':'./data/valid.en',
    'valid_trg_data':'./data/valid.vi',
    'src_lang':'en',
    'trg_lang':'vi',
    'max_strlen':160,
    'batchsize':1500,
    'device':'cuda',
    'd_model': 512,
    'n_layers': 6,
    'heads': 8,
    'dropout': 0.1,
    'lr':0.0001,
    'epochs':5,
    'printevery': 6000,
    'k':5,
}

In [28]:
train_src_data, train_trg_data = read_data(opt['train_src_data'], opt['train_trg_data'])
valid_src_data, valid_trg_data = read_data(opt['valid_src_data'], opt['valid_trg_data'])

SRC, TRG = create_fields(opt['src_lang'], opt['trg_lang'])
train_iter = create_dataset(train_src_data, train_trg_data, opt['max_strlen'], opt['batchsize'], opt['device'], SRC, TRG, istrain=True)
valid_iter = create_dataset(valid_src_data, valid_trg_data, opt['max_strlen'], opt['batchsize'], opt['device'], SRC, TRG, istrain=False)

Creating dataset and Iterator... 
Creating dataset and Iterator... 


In [30]:
src_pad = SRC.vocab.stoi['<pad>']
trg_pad = TRG.vocab.stoi['<pad>']

In [31]:
model = Transformer(len(SRC.vocab), len(TRG.vocab), opt['d_model'], opt['n_layers'], opt['heads'], opt['dropout'])
model = model.to(opt['device'])

# Training model

In [26]:
# Khởi tạo optimizer
optimizer = torch.optim.Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-09)

# Định nghĩa Loss function
criterion = nn.CrossEntropyLoss(ignore_index=trg_pad)

In [27]:
import torch

# Khởi tạo biến để lưu history
train_loss_history = []
valid_loss_history = []

# Hàm lưu mô hình
def save_model(model, epoch, path="model.pth"):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss,
    }, path)
    print(f"Model saved at epoch {epoch+1}!")

best_loss = float('inf')
for epoch in range(opt['epochs']):
    print(f"""*** TRAINING EPOCH {epoch + 1} ***""")
    total_loss = 0
    
    model.train()  # Đảm bảo mô hình ở chế độ huấn luyện
    for i, batch in enumerate(train_iter): 
        loss = step(model, optimizer, batch, criterion)
        total_loss += loss
        
        # In ra kết quả nếu đến thời điểm in
        if (i + 1) % opt['printevery'] == 0:
            avg_loss = total_loss / opt['printevery']
            print(f' \tIter: {i + 1} - Train loss: {avg_loss}')
            total_loss = 0

    # Lưu giá trị train loss cho epoch
    avg_train_loss = total_loss / (i + 1)  
    train_loss_history.append(avg_train_loss)

    # Đánh giá validation loss
    model.eval()  # Đảm bảo mô hình ở chế độ đánh giá
    valid_loss = validiate(model, valid_iter, criterion)
    valid_loss_history.append(valid_loss)

    print("""**** VALIDATION ***""")
    print(f'Epoch: {epoch+1} - Val loss: {valid_loss}')
    
    # Lưu mô hình nếu có sự cải thiện 
    if best_loss > valid_loss:
        best_loss = valid_loss
        save_model(model, epoch, path="model_mt.pth")


*** TRAINING EPOCH 1 ***
 	Iter: 6000 - Train loss: 4.138365945140521
 	Iter: 12000 - Train loss: 3.569727176487446
**** VALIDATION ***
Epoch: 1 - Val loss: 1.5856701850891113
Model saved at epoch 1!
*** TRAINING EPOCH 2 ***
 	Iter: 6000 - Train loss: 3.5314211794137953
 	Iter: 12000 - Train loss: 3.540176963031292
**** VALIDATION ***
Epoch: 2 - Val loss: 1.3945751190185547
Model saved at epoch 2!
*** TRAINING EPOCH 3 ***
 	Iter: 6000 - Train loss: 3.436589025378227
 	Iter: 12000 - Train loss: 3.335584201713403
**** VALIDATION ***
Epoch: 3 - Val loss: 1.1621538400650024
Model saved at epoch 3!
*** TRAINING EPOCH 4 ***
 	Iter: 6000 - Train loss: 3.2003503495057424
 	Iter: 12000 - Train loss: 3.12177966239055
**** VALIDATION ***
Epoch: 4 - Val loss: 1.0084938883781434
Model saved at epoch 4!
*** TRAINING EPOCH 5 ***
 	Iter: 6000 - Train loss: 2.956598842302958
 	Iter: 12000 - Train loss: 2.897527417322
**** VALIDATION ***
Epoch: 5 - Val loss: 0.8986941814422608
Model saved at epoch 5!


# Thực nghiệm

In [28]:
from nltk.corpus import wordnet
import re

def get_synonym(word, SRC):
    syns = wordnet.synsets(word)
    for s in syns:
        for l in s.lemmas():
            if l.name() in SRC.vocab.stoi:  # Kiểm tra nếu từ đồng nghĩa có trong vocab
                return SRC.vocab.stoi[l.name()]
            
    return SRC.vocab.stoi['<unk>']  # Trả về <unk> nếu không tìm thấy từ đồng nghĩa nào hợp lệ

def multiple_replace(dict, text):
  # Create a regular expression  from the dictionary keys
  regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

  # For each match, look-up corresponding value in dictionary
  return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text) 
def init_vars(src, model, SRC, TRG, device, k, max_len):
    """ Tính toán các ma trận cần thiết trong quá trình translation sau khi mô hình học xong
    """
    init_tok = TRG.vocab.stoi['<sos>']
    src_mask = (src != SRC.vocab.stoi['<pad>']).unsqueeze(-2)

    # tính sẵn output của encoder 
    e_output = model.encoder(src, src_mask)
    
    outputs = torch.LongTensor([[init_tok]])
    
    outputs = outputs.to(device)
    
    trg_mask = nopeak_mask(1, device)
    # dự đoán kí tự đầu tiên
    out = model.out(model.decoder(outputs,
    e_output, src_mask, trg_mask))
    out = F.softmax(out, dim=-1)
    
    probs, ix = out[:, -1].data.topk(k)
    log_scores = torch.Tensor([math.log(prob) for prob in probs.data[0]]).unsqueeze(0)
    
    outputs = torch.zeros(k, max_len).long()
    outputs = outputs.to(device)
    outputs[:, 0] = init_tok
    outputs[:, 1] = ix[0]
    
    e_outputs = torch.zeros(k, e_output.size(-2),e_output.size(-1))
   
    e_outputs = e_outputs.to(device)
    e_outputs[:, :] = e_output[0]
    
    return outputs, e_outputs, log_scores

def k_best_outputs(outputs, out, log_scores, i, k):
    
    probs, ix = out[:, -1].data.topk(k)
    log_probs = torch.Tensor([math.log(p) for p in probs.data.view(-1)]).view(k, -1) + log_scores.transpose(0,1)
    k_probs, k_ix = log_probs.view(-1).topk(k)
    
    row = k_ix // k
    col = k_ix % k

    outputs[:, :i] = outputs[row, :i]
    outputs[:, i] = ix[row, col]

    log_scores = k_probs.unsqueeze(0)
    
    return outputs, log_scores

def beam_search(src, model, SRC, TRG, device, k, max_len):    

    outputs, e_outputs, log_scores = init_vars(src, model, SRC, TRG, device, k, max_len)
    eos_tok = TRG.vocab.stoi['<eos>']
    src_mask = (src != SRC.vocab.stoi['<pad>']).unsqueeze(-2)
    ind = None
    for i in range(2, max_len):
    
        trg_mask = nopeak_mask(i, device)

        out = model.out(model.decoder(outputs[:,:i],
        e_outputs, src_mask, trg_mask))

        out = F.softmax(out, dim=-1)
    
        outputs, log_scores = k_best_outputs(outputs, out, log_scores, i, k)
        
        ones = (outputs==eos_tok).nonzero() # Occurrences of end symbols for all input sentences.
        sentence_lengths = torch.zeros(len(outputs), dtype=torch.long).cuda()
        for vec in ones:
            i = vec[0]
            if sentence_lengths[i]==0: # First end symbol has not been found yet
                sentence_lengths[i] = vec[1] # Position of first end symbol

        num_finished_sentences = len([s for s in sentence_lengths if s > 0])

        if num_finished_sentences == k:
            alpha = 0.7
            div = 1/(sentence_lengths.type_as(log_scores)**alpha)
            _, ind = torch.max(log_scores * div, 1)
            ind = ind.data[0]
            break
    
    if ind is None:
        
        length = (outputs[0]==eos_tok).nonzero()[0] if len((outputs[0]==eos_tok).nonzero()) > 0 else -1
        return ' '.join([TRG.vocab.itos[tok] for tok in outputs[0][1:length]])
    
    else:
        length = (outputs[ind]==eos_tok).nonzero()[0]
        return ' '.join([TRG.vocab.itos[tok] for tok in outputs[ind][1:length]])
        
def translate_sentence(sentence, model, SRC, TRG, device, k=5, max_len=50):
    """Dịch câu sử dụng beam search (phiên bản cải tiến)
    
    Args:
        sentence: Câu nguồn cần dịch
        model: Model Transformer đã trained
        SRC: Field xử lý ngôn ngữ nguồn
        TRG: Field xử lý ngôn ngữ đích
        device: 'cuda' hoặc 'cpu'
        k: Số lượng beams (mặc định 5)
        max_len: Độ dài tối đa câu dịch
    
    Returns:
        str: Câu đã dịch
    """
    model.eval()
    
    # 1. Tokenize và numericalize (xử lý OOV)
    tokens = [token.lower() for token in SRC.tokenize(sentence)]
    indexed = [SRC.vocab.stoi.get(token, SRC.vocab.stoi['<unk>']) for token in tokens]
    
    # 2. Tạo tensor đầu vào [1, seq_len]
    src_tensor = torch.LongTensor([indexed]).to(device)
    
    # 3. Beam search
    translation = beam_search(src_tensor, model, SRC, TRG, device, k, max_len)
    
    # 4. Hậu xử lý
    translation = multiple_replace({
        ' ?': '?',
        ' !': '!',
        ' .': '.',
        ' ,': ',',
        ' \'': '\''
    }, translation)
    
    return translation

In [40]:
sentences = [
    "Can I borrow your book?",
    "I want to learn how solve this problem.",
    "Could you help me find the way to the train station?",
    "Thank you very much for your help."
]

# Testing the translation on each sentence
for sentence in sentences:
    trans_sent = translate_sentence(sentence, model, SRC, TRG, device='cuda', k=5)
    print(f"Original: {sentence}")
    print(f"Translated: {trans_sent}")
    print("-" * 50)


Original: Can I borrow your book?
Translated: tôi có thể mượn cuốn sách của bạn được không?
--------------------------------------------------
Original: I want to learn how solve this problem.
Translated: tôi muốn học giải quyết vấn đề này như thế nào.
--------------------------------------------------
Original: Could you help me find the way to the train station?
Translated: bạn có thể giúp tôi tìm đường đến ga xe lửa không?
--------------------------------------------------
Original: Thank you very much for your help.
Translated: cảm ơn các bạn rất nhiều vì sự giúp đỡ của bạn.
--------------------------------------------------


# Evaluate bang BLEU Score

In [32]:
import torch

checkpoint = torch.load('/kaggle/input/weight-mt-scatch-trans/model_mt.pth', 
                       map_location='cuda', 
                       weights_only=True)

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

Transformer(
  (encoder): Encoder(
    (embed): Embedder(
      (embed): Embedding(206392, 512)
    )
    (pe): PositionalEncoder(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (norm_1): Norm()
        (norm_2): Norm()
        (attn): MultiHeadAttention(
          (q_linear): Linear(in_features=512, out_features=512, bias=True)
          (k_linear): Linear(in_features=512, out_features=512, bias=True)
          (v_linear): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (out): Linear(in_features=512, out_features=512, bias=True)
        )
        (ff): FeedForward(
          (linear_1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear_2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (dropout_1): Dropout(p=0.1, inplace=False)
        (dropout_2): Dropo

In [37]:
!pip install -q evaluate rouge_score

  Preparing metadata (setup.py) ... done


In [42]:
def read_data(src_file, trg_file):
    """Đọc dữ liệu từ file nguồn và file đích."""
    try:
        with open(src_file, 'r', encoding='utf-8') as f_src, \
             open(trg_file, 'r', encoding='utf-8') as f_trg:
            # Đọc từng dòng, loại bỏ khoảng trắng thừa và bỏ qua dòng trống
            src_data = [line.strip() for line in f_src if line.strip()]
            trg_data = [line.strip() for line in f_trg if line.strip()]
        if len(src_data) != len(trg_data):
             print(f"Cảnh báo: Số dòng không khớp giữa {src_file} ({len(src_data)}) và {trg_file} ({len(trg_data)}).")
             # Giới hạn theo số dòng ít hơn để tránh lỗi zip sau này
             min_len = min(len(src_data), len(trg_data))
             src_data = src_data[:min_len]
             trg_data = trg_data[:min_len]
        return src_data, trg_data
    except FileNotFoundError as e:
        print(f"Lỗi: Không tìm thấy file: {e}")
        return None, None
    except Exception as e:
        print(f"Lỗi khi đọc file: {e}")
        return None, None

def evaluate_model(model, SRC, TRG, device, valid_src_path, valid_trg_path, batch_size=32, max_sentences=None):
    print("--- Bắt đầu quá trình đánh giá ---")

    # Tải metrics
    try:
        bleu_metric = evaluate.load("bleu")
        rouge_metric = evaluate.load("rouge")
        print("Đã tải xong metrics BLEU và ROUGE.")
    except Exception as e:
        print(f"Lỗi khi tải metric từ thư viện 'evaluate': {e}")
        print("Hãy đảm bảo bạn đã cài đặt: pip install evaluate sacrebleu rouge_score")
        return None

    # Đọc dữ liệu validation
    print(f"Đang đọc dữ liệu validation từ:\n- Nguồn: {valid_src_path}\n- Đích: {valid_trg_path}")
    src_sentences, ref_sentences = read_data(valid_src_path, valid_trg_path)

    if src_sentences is None or ref_sentences is None:
        print("Không thể tiếp tục đánh giá do lỗi đọc file.")
        return None

    print(f"Đã đọc thành công {len(src_sentences)} câu.")

    # Giới hạn số câu nếu cần
    if max_sentences is not None and max_sentences < len(src_sentences):
        print(f"Giới hạn đánh giá ở {max_sentences} câu đầu tiên.")
        src_sentences = src_sentences[:max_sentences]
        ref_sentences = ref_sentences[:max_sentences]

    candidates = [] # Lưu các câu dịch được bởi model
    references = [] # Lưu các câu tham chiếu (có thể là list of lists cho BLEU)

    print(f"Bắt đầu dịch {len(src_sentences)} câu validation...")
    model.eval() # Đảm bảo model ở chế độ đánh giá

    # --- Vòng lặp dịch từng câu ---
    for i in tqdm(range(len(src_sentences)), desc="Đang dịch"):
        src_sent = src_sentences[i]
        ref_sent = ref_sentences[i]

        try:
            # Gọi hàm dịch của bạn
            translated_sent = translate_sentence(src_sent, model, SRC, TRG, device, k=opt.get('k', 5), max_len=opt.get('max_strlen', 50)) # Lấy k và max_len từ opt nếu có
            candidates.append(translated_sent)
            references.append(ref_sent) # Lưu ref dạng string đơn giản

        except Exception as e:
            print(f"\nCảnh báo: Lỗi khi dịch câu {i+1}: '{src_sent[:50]}...'")
            print(f"  Lỗi: {e}")
            # Quyết định xử lý lỗi:
            # 1. Bỏ qua câu này: continue
            # 2. Thêm kết quả rỗng để giữ đúng số lượng:
            candidates.append("") # Thêm chuỗi rỗng
            references.append(ref_sent) # Vẫn thêm ref tương ứng
            # 3. Dừng hẳn: raise e

    print("Đã dịch xong. Bắt đầu tính toán điểm...")

    # Định dạng references cho BLEU của thư viện evaluate
    # Nếu chỉ có 1 ref cho mỗi câu nguồn:
    references_for_bleu = [[ref] for ref in references]

    # Tính toán điểm
    try:
        bleu_score = bleu_metric.compute(predictions=candidates, references=references_for_bleu)
        # Metric ROUGE thường chấp nhận list of strings cho references
        rouge_score = rouge_metric.compute(predictions=candidates, references=references)

        print("\n--- Kết quả đánh giá ---")
        # BLEU score thường hiển thị dạng 0-100
        print(f"BLEU: {bleu_score['bleu'] * 100:.2f}")
        # ROUGE scores (thường là F1-score)
        print(f"ROUGE-1: {rouge_score['rouge1']:.4f}")
        print(f"ROUGE-2: {rouge_score['rouge2']:.4f}")
        print(f"ROUGE-L: {rouge_score['rougeL']:.4f}") # Longest common subsequence
        print(f"ROUGE-Lsum: {rouge_score['rougeLsum']:.4f}") # Lsum tóm tắt các câu dài
        print("------------------------")

        # Trả về dictionary chứa các điểm (BLEU dạng 0-1)
        results = {
            'bleu': bleu_score['bleu'],
            'rouge1': rouge_score['rouge1'],
            'rouge2': rouge_score['rouge2'],
            'rougeL': rouge_score['rougeL'],
            'rougeLsum': rouge_score['rougeLsum']
        }
        return results

    except Exception as e:
        print(f"\nLỗi trong quá trình tính toán điểm: {e}")
        print("Kiểm tra định dạng của candidates và references.")
        return None

# Lấy đường dẫn file validation từ dict 'opt' của bạn
valid_en_path = opt.get('valid_src_data')
valid_vi_path = opt.get('valid_trg_data')
device = opt.get('device', 'cuda' if torch.cuda.is_available() else 'cpu') # Lấy device từ opt hoặc tự xác định

if valid_en_path and valid_vi_path:
    evaluation_scores = evaluate_model(
        model=model,
        SRC=SRC,
        TRG=TRG,
        device=device,
        valid_src_path=valid_en_path,
        valid_trg_path=valid_vi_path,
        max_sentences=None # Đặt None để đánh giá toàn bộ tập validation
    )

    if evaluation_scores:
        print("\nĐiểm đánh giá chi tiết:")
        print(evaluation_scores)
else:
    print("Lỗi: Không tìm thấy đường dẫn file validation ('valid_src_data', 'valid_trg_data') trong biến 'opt'.")


--- Bắt đầu quá trình đánh giá ---
Đã tải xong metrics BLEU và ROUGE.
Đang đọc dữ liệu validation từ:
- Nguồn: ./data/valid.en
- Đích: ./data/valid.vi
Đã đọc thành công 500 câu.
Bắt đầu dịch 500 câu validation...


Đang dịch: 100%|██████████| 500/500 [01:49<00:00,  4.56it/s]


Đã dịch xong. Bắt đầu tính toán điểm...

--- Kết quả đánh giá ---
BLEU: 29.37
ROUGE-1: 0.7876
ROUGE-2: 0.6570
ROUGE-L: 0.7442
ROUGE-Lsum: 0.7442
------------------------

Điểm đánh giá chi tiết:
{'bleu': 0.29372343076727364, 'rouge1': 0.7875562682149484, 'rouge2': 0.6570476672441349, 'rougeL': 0.744182961913248, 'rougeLsum': 0.7442248957555513}
